<title> Building Calnet Example</title>
# How to build Calnet
written by Aric Sanders, 02/2016

A general description of how to build calnet. 

## 1. Make the skeleton 

   1. run "django-admin startproject MyProject" from a command prompt
   2. move into the directory containing manage.py 
        + i.e. run "cd MyProject"
   3. run "python manage.py startapp AppName" for each app
        + if you have multiple python installs you can replace python with the path to the bin or exe
   4. make some standard directories your favorite way
        + django name spaces its static directories as AppName/static/AppName/ the same with template directories
        
<em>Here we use a python script to create the directories we need</em>

In [1]:
def write_project_urls(app_names):
    """Writes a basic urls.py for a django-project 
    given the app names"""
    urls_imports=['from django.conf.urls import include, url\n',
                 'from django.contrib import admin\n']
    
    url_pattern_lines=['urlpatterns= [\n',
                       "  url(r'^admin/', admin.site.urls),\n"
                      ]
    for app_name in app_names:
        url_pattern="  url(r'^%s/',include('%s.urls')),\n"%(app_name,app_name)
        url_pattern_lines.append(url_pattern)
    url_pattern_lines.append('  ]')

    out_file=open('urls.py','w')
    for line in urls_imports:
        out_file.write(line)
    for line in url_pattern_lines:
        out_file.write(line)
    out_file.close()
    
def write_app_urls(app_name):
    """Writes a basic urls.py for a django-app"""
    # Write the import string
    urls_imports=['from django.conf.urls import url\n',
                 'from . import views\n']
    # write the app_name = line
    app_name_line="app_name= '%s'\n"%app_name
    # write the url patterns
    url_pattern_lines=['urlpatterns= [\n',
                       "  url(r'^$',views.index,name='index'),\n",
                      '  ]\n']
    # open the file and do the work
    out_file=open('urls.py','w')
    for line in urls_imports:
        out_file.write(line)
    out_file.write(app_name_line)
    for line in url_pattern_lines:
        out_file.write(line)
    out_file.close()

In [3]:
import os
try:
    # set the top most directory
    current_directory=os.getcwd() # Should change this if it needs to be
    # name the project
    project_name='Calnet'
    #name the apps, use a general naming scheme. Here I have used CamelCase and Nouns
    app_names=['Help','Repository','Preferences','ProjectTracker','CheckStandard',
               'Uncertainties','Measurement']
    # run a system command to create the project
    os.system('django-admin startproject %s'%project_name)
    # move into the directory with manage.py in it
    os.chdir(os.path.join(current_directory,'%s'%project_name))
    # keep the current directory
    current_directory=os.getcwd()
    manage_directory=os.getcwd() # we will want this directory for a lot of things
    # write a urls for the project
    os.chdir(os.path.join(manage_directory,'%s'%project_name))
    write_project_urls(app_names)
    os.chdir(manage_directory)
    # loop through the apps creating them 
    for app in app_names:
        os.system('python manage.py startapp %s'%app)   
    # Now write the urls.py for each app
    for app in app_names:
        os.chdir(os.path.join(manage_directory,'%s'%app))
        write_app_urls(app)
    # Once the apps are made, make static and 
    # template directories in each app for {{ static files }}
    # Note this could be combined with the 'startapp' loop
    for app in app_names:
        # static directory is redundant for makedirs, but retained for clarity
        # These directories are in the location django will look for them
        static_directory=os.path.join(current_directory,'%s'%app,'static','%s'%app)
        template_directory=os.path.join(current_directory,'%s'%app,'templates','%s'%app)
        # This could be a list of exec commands
        img_directory=os.path.join(current_directory,'%s'%app,'static','%s'%app,'img')
        js_directory=os.path.join(current_directory,'%s'%app,'static','%s'%app,'js')
        css_directory=os.path.join(current_directory,'%s'%app,'static','%s'%app,'css')
        # directories specific to my way of doing help or documentation
        jupyter_directory=os.path.join(current_directory,'%s'%app,'static','%s'%app,'jupyter')
        html_directory=os.path.join(current_directory,'%s'%app,'static','%s'%app,'html')
        cache=os.path.join(current_directory,'%s'%app,'static','%s'%app,'cache')
        # help application
        if app in ['Help','help','doc','docs','documentation']:
            os.makedirs(static_directory)
            os.makedirs(template_directory)
            os.makedirs(img_directory)
            os.makedirs(js_directory)
            os.makedirs(css_directory)
            os.makedirs(cache)
            os.makedirs(jupyter_directory)
            os.makedirs(html_directory)
            
        else:
            os.makedirs(static_directory)
            os.makedirs(template_directory)
            os.makedirs(img_directory)
            os.makedirs(js_directory)
            os.makedirs(css_directory)
            os.makedirs(cache)
except:raise

In [4]:
manage_directory

'C:\\Users\\arics\\Jupyter-Notebooks\\Calnet'

## 2. Define the models that you will need for each app
+ If you don't know yet don't worry
    1. Go to the App/models.py python file and add a model
        + Each model is a class that is derived from models.Model
        + Each model has Fields that are the columns
        + By Adding the \__str__ method you control how django displays the model
        + If you want to define a general "schema" or model form then use an inner class Meta, with the attribute abstract = True, I think these will be at pyMeasure.Code.DataHandlers.AbstractDjangoModels
        ```python
        class LogSchema(django.db.models.Model):
        """ A class schema that holds an index and a date """
        # This is an explicit replica of the auto generated field id
        index=IntegerField('Entry Number')
        # I use a date time field to time stamp entries
        # if you want to fix it to the point the entry is added then use ,auto_now_add=True
        date=DateTimeField('Date of entry')
        
        # The meta class is anything that is not a field. Setting proxy or abstract to True are special cases
        class Meta:
            abstract=True
        def __str__(self):
            """ A default setting override in new logs"""
            string_text="Entry %s at %s"%(self.index,str(self.date))
            return string_text
            ```
        + Now the actual table is the class that inherits from it
           ```python
           class UserLog(LogSchema):
           """ A Log that has a user and text field associated with it"""
           # I am not sure if this is supposed to be user or settings.AUTH_USER_MODEL
           user=models.ForeignKey(User,models.SET_NULL,blank=True,null=True,)
           # Now a text area or char field your choice, could be a summernote area also
           note=TextField('Note')
           def __str__(self):
               """ Display as a string, in this case Note number 1 left by user on date """
               # Todo replace with "".format()
               string_text="Note number %s left by %s %s on %s"%(str(self.index),User.first_name, User.last_name, str(self.date))
               return string_text
               ```
               

## 3. Register each model with App using the admin

1. Go to each Calnet/Help/admin.py and register all the models

```python
from django.contrib import admin

from .models import UserLog

admin.site.register(UserLog)
```

## 4. Register the Apps with the site using settings.py

1. Open Calnet/Calnet/settings.py and change INSTALLED_APPS as follows:

```python

    INSTALLED_APPS = [
    'polls.apps.PollsConfig',
    'django.contrib.admin',
    'django.contrib.auth',
    'django.contrib.contenttypes',
    'django.contrib.sessions',
    'django.contrib.messages',
    'django.contrib.staticfiles',
    'Help',
    'Repository'
    'Preferences'
    'ProjectTracker'
    'CheckStandard'
    'Uncertainties',
    'Measurement'
    ]
```

## 5. Make Migrations

1. In order to update the models in the database and with the admin site you have to run makemigrations and migrate.

In [ ]:
# make sure you are in the manage_directory
#import os if a different session from the skeleton generation
os.chdir(manage_directory)
os.system('python manage.py makemigrations')
os.system('python manage.py migrate')

## 6. Create a superuser account

1. In order to create a super user account use the shell to run python manage.py createsuperuser

```python
# This should work, not sure if the shell will stop
os.system('python manage.py createsuperuser')
```


## 7. Make urls.py in the site directory point to the App urls and admin

1. Go to the MySite/urls.py and add each App

```python
from django.conf.urls import include, url
from django.contrib import admin

urlpatterns = [
    url(r'^App/', include('App.urls')),
    url(r'^admin/', admin.site.urls),
]
```

## 8. Make the App urls point to views

1. Open each App/urls.py and put a pattern that points to a view

## 9. Define the views in App/view.py

1. open each Calnet/App/views.py and add a function that returns a view, use templates.

## 10. Define the templates in App/templates/App

## 11. Add all the needed javascript, images and css to App/static/App/